<a href="https://colab.research.google.com/github/reedamroue/EIP/blob/main/Workbook_EIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import re
!pip install validators
!pip install nltk
import nltk
import validators
import sys
from urllib.parse import urlparse
!pip install wget
from urllib.request import urlopen
import urllib3
!pip install PyPDF4
!pip install PyMuPDF
import PyPDF4
import urllib.request 
import fitz  # this is pymupdf
import pandas as pd

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=a29050b6d23ea329b4cc2d7f37fcd371dc2d620a58f61e9a1b2e707cabbecfe2
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-cp37-none-any.whl size=61249 sha256=fa74b299873ce34f1aed5b02f2d53e2ee5ef7cb17f330e4f2d2fbc751eec32b2
  Stored in directory: /root/.cache/pip/wheels/eb/4f/15/c64d533cb496fd874f56045fe30e8cc0ac59f99ecdd718040d
Successfully built PyPDF4
     |████████████████████████████████| 6.4MB 23.7MB/s 


In [2]:
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

In [3]:
def check_validity(my_url):
    try:
        urlopen(my_url)
    except IOError:
        print ("Please input a valid URL!")
        sys.exit()


def get_pdfs(my_url):
    links = []
    links_final = []
    html = urlopen(my_url).read()
    html_page = BeautifulSoup(html, features="lxml") 
    og_url = html_page.find("meta",  property = "og:url")
    base = urlparse(my_url)
    ##print("base",base)
    for link in html_page.find_all('a'):
        current_link = link.get('href')
        if current_link.endswith('pdf'):
            if og_url:
                ##print("currentLink",current_link)
                links.append(og_url["content"] + current_link)
            else:
                links.append(base.scheme + "://" + base.netloc + current_link)

    for link in links:
        try: 
            wget.download(link)
        except:
            ##print(" \n \n Unable to Download A File \n")
            links.remove(link)
    for link in links:
      http = urllib3.PoolManager()
      error = http.request("GET", link)
      if (error.status != 404):
        links_final.append(link)
    count = 0
    for link in links_final:
      download_file(link, "pdf" + str(count))
      count+=1
    my_df = pd.DataFrame(links_final, columns = ['PDF Links'])
    my_df.to_csv('pdf_links', index = False)

In [4]:
get_pdfs('https://cnds.jacobs-university.de/courses/os-2019/')

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is

In [5]:
mypdf = open('samp_elec.pdf', mode='rb')

In [6]:
pdf_document = PyPDF4.PdfFileReader(mypdf)

In [7]:
pdf_document.numPages

18

In [8]:
def get_text_pdf_page(pdf, page):
  with fitz.open(pdf) as doc:
      text = ""
      text += doc[page - 1].getText()
  ##print(text)
  return text

def get_text_pdf_full(pdf):
  mypdf = open(pdf, mode='rb')
  pdf_document = PyPDF4.PdfFileReader(mypdf)
  pages = pdf_document.numPages
  with fitz.open(pdf) as doc:
      text = ""
      for i in range(0, pages):
        text += doc[i].getText()
        text += '\n'
        text += 'NEW_PAGE'
        text += '\n'
  return text

In [9]:
def sent_pg_labeler(text):
  output = ""
  lst_out = []
  pages = text.split("\nNEW_PAGE\n")
  index = 1
  for p in pages:
    lst_out.append(p.replace('. ', ' (pg# ' + str(index) + ').'))
    index += 1
  for x in lst_out:
    output += x
  return output

In [10]:
text = get_text_pdf_page('samp_elec.pdf', 1)
text = text.replace('\n','')
text

'  1  BEFORE THE FLORIDA PUBLIC SERVICE COMMISSION   In re:  Petition of Tampa Electric Company )  DOCKET NO.________________ for Approval of Electric Vehicle Charging ) Pilot Program               ) ___________________________________ )  FILED:  September 25, 2020   PETITION OF TAMPA ELECTRIC COMPANY FOR APPROVAL OF ELECTRIC VEHICLE CHARGING PILOT PROGRAM  Tampa Electric Company ("Tampa Electric" or the “company"), pursuant to Sections 120.521 and 366.04, Florida Statutes petitions the Florida Public Service Commission (the “Commission") for approval of an electric vehicle charging pilot program, and as grounds therefore, states: I. Preliminary Information 1. The Petitioner’s name and address are: Tampa Electric Company 702 North Franklin Street Tampa, Florida 33602  2. Tampa Electric is an investor owned electric utility subject to the Commission\'s jurisdiction pursuant to Chapter 366, Florida Statutes.  Tampa Electric serves retail customers in Hillsborough and portions of Polk, Pi

In [11]:
text1 = get_text_pdf_full('samp_elec.pdf')
text1 = text1.replace('\n','')
text1

'  1  BEFORE THE FLORIDA PUBLIC SERVICE COMMISSION   In re:  Petition of Tampa Electric Company )  DOCKET NO.________________ for Approval of Electric Vehicle Charging ) Pilot Program               ) ___________________________________ )  FILED:  September 25, 2020   PETITION OF TAMPA ELECTRIC COMPANY FOR APPROVAL OF ELECTRIC VEHICLE CHARGING PILOT PROGRAM  Tampa Electric Company ("Tampa Electric" or the “company"), pursuant to Sections 120.521 and 366.04, Florida Statutes petitions the Florida Public Service Commission (the “Commission") for approval of an electric vehicle charging pilot program, and as grounds therefore, states: I. Preliminary Information 1. The Petitioner’s name and address are: Tampa Electric Company 702 North Franklin Street Tampa, Florida 33602  2. Tampa Electric is an investor owned electric utility subject to the Commission\'s jurisdiction pursuant to Chapter 366, Florida Statutes.  Tampa Electric serves retail customers in Hillsborough and portions of Polk, Pi

In [12]:
text5 = get_text_pdf_full('duke_florida.pdf')
text5 = text5.replace('\n','')
text5

'FLORIDA VIA ELECTRONIC FILING Adam J. T eitzman, Commission Clerk Florida Public Service Commission 2540 Shumard Oak Boulevru:d Tallahassee, Florida 32399-0850 December 18, 2020 FILED 12/18/2020 DOCUMENT NO. 13637-2020 FPSC - COMMISSION CLERK Matthew R. Bernier ASSOCIATE GENERAL COUNSEL Re: Duke Energy Florida, LLC\'s Application for limited proceeding to approve 2017 second revised and restated settlement agreement, including certain rate adjustments; Docket No. 20170183-EI Dear Mr. Teitzman: Enclosed for filing on behalf of Duke Energy Florida, LLC ("DEF") is DEF\'s Electric Vehicle Charging Station Pilot Program - 3rd Annual Report (December 2020), in accordance with Paragraph 17.f.ii of the 2017 Second Revised and Restated Settlement Agreement, which was approved in Order No. PSC-2017-0451-AS-EU, dated November 20, 2017. Thank you for your assistance in this matter. Please feel free to call me at (850) 521-1428 should you have any questions concerning this filing. MRB/cmk/cmw Encl

In [13]:
text6 = get_text_pdf_full('duke_carolinas.pdf')
text6 = text6.replace('\n','')
text6

'STATE OF NORTH CAROLINA UTILITIES COMMISSION RALEIGH DOCKET NO. E-2, SUB 1197 DOCKET NO. E-7, SUB 1195 In the Matter of Application by Duke Energy Carolinas, LLC, and Duke Energy Progress, LLC, for Approval of Proposed Electric Transportation Pilot  ) ) )   ORDER APPROVING ELECTRIC TRANSPORTATION PILOT, IN PART  HEARD: Thursday, November 21, 2019, at 10:30 a.m., in Commission Hearing Room 2115, Dobbs Building, 430 North Salisbury Street, Raleigh, North Carolina BEFORE: Chair Charlotte A. Mitchell, Presiding; Commissioners ToNola D. Brown-Bland, Lyons Gray, Daniel G. Clodfelter, Kimberly W. Duffley, and Jeffrey A. Hughes APPEARANCES: For Duke Energy Carolinas, LLC, and Duke Energy Progress, LLC: Kendrick C. Fentress, Associate General Counsel, Duke Energy Corporation, 410 South Wilmington Street, NCRH 20/Post Office Box 1551, Raleigh, North Carolina 27602 Robert W. Kaylor, Law Office of Robert W. Kaylor, P.A., 353 East Six Forks Road, Suite 260, Raleigh, North Carolina 27609 Dwight All

In [14]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [15]:
article = nlp(text1)
##items = [article.ents]
items = [x for x in article.ents]
items = filter(lambda x: x.label_ == 'DATE', items)

items1 = []
##Counter(labels)
Counter(items).most_common(10)
# for x in article.ents:
#   if x.label_ == 'ORG':
#     items1.append(x)
  
##Counter(items1).most_common(1)

[(September 25, 2020, 1),
 (28-106.201, 1),
 (four years, 1),
 (the coming years, 1),
 (the same year, 1),
 (2011, 1),
 (2017, 1),
 (June of 2020, 1),
 (120V, 1),
 (approximately 12 months, 1)]

In [16]:
from spacy.matcher import Matcher

def get_company_name(pdf1):
  i = get_text_pdf_full(pdf1)
  i = i.replace('\n','')
  doc = nlp(i)

  list1 = []

  output = []
  output1 = []
  output2 = []

  pattern = [{"LOWER": "petition"}, {"LOWER": "of"}, {"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}]
  ##pattern1 = [{"LOWER": "by"}, {"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}]
  pattern3 = [{"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}, {"IS_PUNCT": True}, {"LOWER": "llc"}]
  pattern2 = [{"LOWER": "re"}, {"IS_PUNCT": True}, {"ENT_TYPE": "ORG"}, {"ENT_TYPE": "ORG"}]

  matcher = Matcher(nlp.vocab)
  matcher.add('org_only', None, pattern)
  ##matcher.add('org_only1', None, pattern1)
  matcher.add('org_only2', None, pattern2)
  matcher.add('org_only3', None, pattern3)
  matches = matcher(doc)
  for match_id, start, end in matches:
      list1.append(doc[start:end])
  for item in list1:
    output.append(item.text.lower())

  output = set(output)
  output = list(output)
  for x in output:
    output1.append(x.replace('re: ', '').replace('petition of ', '').replace('by', ''))
  for x in output1:
    output2.append(x.replace("new_page", ""))
  if len(output2) > 1:
    map = []
    for x in output2:
      num = i.lower().count(x)
      map.append((x, num))
    lst = len(map) 
    for i in range(0, lst):          
      for j in range(0, lst-i-1): 
          if (map[j][1] > map[j + 1][1]): 
              temp = map[j] 
              map[j]= map[j + 1] 
              map[j + 1]= temp 
    return map[lst - 1][0]
  else:
    return output2[0]

In [17]:
import random
from spacy.util import compounding
from spacy.util import minibatch

def get_date_full_code(o2, sentence):
  map = []
  for x in o2:
    num = sentence.lower().count(x)
    map.append((x, num))

  lst = len(map) 
  for i in range(0, lst):          
    for j in range(0, lst-i-1): 
        if (map[j][1] > map[j + 1][1]): 
            temp = map[j] 
            map[j]= map[j + 1] 
            map[j + 1]= temp 
  return map[lst - 1][0]

def get_month(sentence):

  map = []
  num_sep = sentence.lower().split().count("september")
  map.append(("September", num_sep))
  num_oct = sentence.lower().split().count("october")
  map.append(("October", num_oct))
  num_nov = sentence.lower().split().count("november")
  map.append(("November", num_nov))
  num_dec = sentence.lower().split().count("december")
  map.append(("December", num_dec))
  num_jan = sentence.lower().split().count("january")
  map.append(("January", num_jan))
  num_feb = sentence.lower().split().count("february")
  map.append(("February", num_feb))
  num_mar = sentence.lower().split().count("march")
  map.append(("March", num_mar))
  num_apr = sentence.lower().split().count("april")
  map.append(("April", num_apr))
  num_may = sentence.lower().split().count("may, ")
  map.append(("May", num_may))
  num_jun = sentence.lower().split().count("june")
  map.append(("June", num_jun))
  num_jul = sentence.lower().split().count("july")
  map.append(("July", num_jul))
  num_aug = sentence.lower().split().count("august")
  map.append(("August", num_aug))

  lst = len(map) 
  for i in range(0, lst):          
    for j in range(0, lst-i-1): 
        if (map[j][1] > map[j + 1][1]): 
            temp = map[j] 
            map[j]= map[j + 1] 
            map[j + 1]= temp 
  return map[lst - 1][0]

def get_date(pdf1):

  i = get_text_pdf_full(pdf1)
  i = i.replace('\n','')
  doc = nlp(i)

  list1 = []

  output = []

  output2 = []

  pattern = [{"IS_ALPHA": True}, {"IS_DIGIT": True}, {"IS_PUNCT": True}, {"IS_DIGIT": True}]

  matcher = Matcher(nlp.vocab)

  matcher.add('date_only', [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
      list1.append(doc[start:end])
  for item in list1:
    output.append(item.text.lower())

  output = set(output)
  output = list(output)
  month = get_month(i).lower()
  for x in output:
    if x.lower().startswith(month):
      output2.append(x)
  if len(output2) > 1:
    output2 = [get_date_full_code(output2, i)]
  # elmt = output.pop().replace('filed: ', '')


  return str(output2[0])

In [18]:
get_company_name('duke_florida.pdf')

'duke energy'

In [19]:
get_date('samp_elec.pdf')

'september 25, 2020'

In [20]:
import gensim
from gensim.summarization import summarize

summarize(text1, 0.1)

'Under this Pilot, Tampa Electric will purchase, install, own, and maintain approximately 200 PEV charging ports within the company’s service territory.\nThe PEV charging ports will be deployed at Tampa Electric customer locations in five different market segments: (1) workplaces; (2) public/retail; (3) multi-unit dwellings; (4) income qualified; and (5) government.\nTampa Electric will pay up to $5,000 per charge port towards the cost of installation of the charging equipment for workplace, public/retail and multi-dwelling segments, and the full cost of installation for income qualified and government locations.\nThe objectives of the Pilot are to provide increased customer confidence in the availability of public charging locations, thereby supporting PEV adoption, and to provide Tampa Electric with valuable experience with public PEV charging infrastructure development and PEV charging load profile data.\nApproval of Tampa Electric’s proposed Pilot will enable the company to manage 

In [21]:
def get_email_addresses(pdf1):
    string = get_text_pdf_full(pdf1)
    string = string.replace('\n','')
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)
    
def get_phone_numbers(pdf1):
    string = get_text_pdf_full(pdf1)
    string = string.replace('\n','')
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', num) for num in phone_numbers]

In [22]:
get_email_addresses('samp_elec.pdf')

['jbeasley@ausley.com',
 'regdept@tecoenergy.com',
 'jwahlen@ausley.com',
 'mmeans@ausley.com',
 'jbeasley@ausley.com',
 'jwahlen@ausley.com',
 'mmeans@ausley.com']

In [23]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# def _create_frequency_table(text_string) -> dict:
#     """
#     we create a dictionary for the word frequency table.
#     For this, we should only use the words that are not part of the stopWords array.
#     Removing stop words and making frequency table
#     Stemmer - an algorithm to bring words to its root word.
#     :rtype: dict
#     """
#     stopWords = set(stopwords.words("english"))
#     words = word_tokenize(text_string)
#     ps = PorterStemmer()

#     freqTable = dict()
#     for word in words:
#         word = ps.stem(word)
#         if word in stopWords:
#             continue
#         if word in freqTable:
#             freqTable[word] += 1
#         else:
#             freqTable[word] = 1

#     return freqTable


# def _score_sentences(sentences, freqTable) -> dict:
#     """
#     score a sentence by its words
#     Basic algorithm: adding the frequency of every non-stop word in a sentence divided by total no of words in a sentence.
#     :rtype: dict
#     """

#     sentenceValue = dict()

#     for sentence in sentences:
#         word_count_in_sentence = (len(word_tokenize(sentence)))
#         word_count_in_sentence_except_stop_words = 0
#         for wordValue in freqTable:
#             if wordValue in sentence.lower():
#                 word_count_in_sentence_except_stop_words += 1
#                 if sentence[:10] in sentenceValue:
#                     sentenceValue[sentence[:10]] += freqTable[wordValue]
#                 else:
#                     sentenceValue[sentence[:10]] = freqTable[wordValue]

#         if sentence[:10] in sentenceValue:
#             sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] / word_count_in_sentence_except_stop_words

#         '''
#         Notice that a potential issue with our score algorithm is that long sentences will have an advantage over short sentences. 
#         To solve this, we're dividing every sentence score by the number of words in the sentence.
        
#         Note that here sentence[:10] is the first 10 character of any sentence, this is to save memory while saving keys of
#         the dictionary.
#         '''

#     return sentenceValue


# def _find_average_score(sentenceValue) -> int:
#     """
#     Find the average score from the sentence value dictionary
#     :rtype: int
#     """
#     sumValues = 0
#     for entry in sentenceValue:
#         sumValues += sentenceValue[entry]

#     # Average value of a sentence from original text
#     average = (sumValues / len(sentenceValue))

#     return average


# def _generate_summary(sentences, sentenceValue, threshold):
#     sentence_count = 0
#     summary = ''

#     for sentence in sentences:
#         if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
#             summary += " " + sentence
#             sentence_count += 1

#     return summary


# def run_summarization(text):
#     # 1 Create the word frequency table
#     freq_table = _create_frequency_table(text)

#     '''
#     We already have a sentence tokenizer, so we just need 
#     to run the sent_tokenize() method to create the array of sentences.
#     '''

#     # 2 Tokenize the sentences
#     sentences = sent_tokenize(text)

#     # 3 Important Algorithm: score the sentences
#     sentence_scores = _score_sentences(sentences, freq_table)

#     # 4 Find the threshold
#     threshold = _find_average_score(sentence_scores)

#     # 5 Important Algorithm: Generate the summary
#     summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)

#     return summary

# run_summarization(text1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
import logging
import textwrap
from gensim.summarization.pagerank_weighted import pagerank_weighted as _pagerank
from gensim.summarization.textcleaner import clean_text_by_sentences as _clean_text_by_sentences
from gensim.summarization.commons import build_graph as _build_graph
from gensim.summarization.commons import remove_unreachable_nodes as _remove_unreachable_nodes
from gensim.summarization.bm25 import get_bm25_weights as _bm25_weights
from gensim.corpora import Dictionary
from math import log10 as _log10
from six.moves import xrange


INPUT_MIN_LENGTH = 10

WEIGHT_THRESHOLD = 1.e-3

logger = logging.getLogger(__name__)


def _set_graph_edge_weights(graph):
    documents = graph.nodes()
    weights = _bm25_weights(documents)

    for i in xrange(len(documents)):
        for j in xrange(len(documents)):
            if i == j or weights[i][j] < WEIGHT_THRESHOLD:
                continue

            sentence_1 = documents[i]
            sentence_2 = documents[j]

            edge_1 = (sentence_1, sentence_2)
            edge_2 = (sentence_2, sentence_1)

            if not graph.has_edge(edge_1):
                graph.add_edge(edge_1, weights[i][j])
            if not graph.has_edge(edge_2):
                graph.add_edge(edge_2, weights[j][i])

    if all(graph.edge_weight(edge) == 0 for edge in graph.edges()):
        _create_valid_graph(graph)


def _create_valid_graph(graph):
    nodes = graph.nodes()

    for i in xrange(len(nodes)):
        for j in xrange(len(nodes)):
            if i == j:
                continue

            edge = (nodes[i], nodes[j])

            if graph.has_edge(edge):
                graph.del_edge(edge)

            graph.add_edge(edge, 1)


def _get_doc_length(doc):
    return sum([item[1] for item in doc])


def _get_similarity(doc1, doc2, vec1, vec2):
    numerator = vec1.dot(vec2.transpose()).toarray()[0][0]
    length_1 = _get_doc_length(doc1)
    length_2 = _get_doc_length(doc2)

    denominator = _log10(length_1) + _log10(length_2) if length_1 > 0 and length_2 > 0 else 0

    return numerator / denominator if denominator != 0 else 0


def _build_corpus(sentences):
    split_tokens = [sentence.token.split() for sentence in sentences]
    dictionary = Dictionary(split_tokens)
    return [dictionary.doc2bow(token) for token in split_tokens]


def _get_important_sentences(sentences, corpus, important_docs):
    hashable_corpus = _build_hasheable_corpus(corpus)
    sentences_by_corpus = dict(zip(hashable_corpus, sentences))
    return [sentences_by_corpus[tuple(important_doc)] for important_doc in important_docs]


def _get_sentences_with_word_count(sentences, word_count):
    length = 0
    selected_sentences = []

    for sentence in sentences:
        words_in_sentence = len(sentence.text.split())

        if abs(word_count - length - words_in_sentence) > abs(word_count - length):
            return selected_sentences

        selected_sentences.append(sentence)
        length += words_in_sentence

    return selected_sentences


def _extract_important_sentences(sentences, corpus, important_docs, word_count):
    important_sentences = _get_important_sentences(sentences, corpus, important_docs)

    return important_sentences if word_count is None else _get_sentences_with_word_count(important_sentences, word_count)


def _format_results(extracted_sentences, split):
    if split:
        return [sentence.text for sentence in extracted_sentences]
    return "\n".join([sentence.text for sentence in extracted_sentences])


def _build_hasheable_corpus(corpus):
    return [tuple(doc) for doc in corpus]


def summarize_corpus(corpus, ratio=0.2):

    hashable_corpus = _build_hasheable_corpus(corpus)

    if len(corpus) == 0:
        logger.warning("Input corpus is empty.")
        return

    if len(corpus) < INPUT_MIN_LENGTH:
        logger.warning("Input corpus is expected to have at least " + str(INPUT_MIN_LENGTH) + " documents.")

    graph = _build_graph(hashable_corpus)
    _set_graph_edge_weights(graph)
    _remove_unreachable_nodes(graph)

    pagerank_scores = _pagerank(graph)

    hashable_corpus.sort(key=lambda doc: pagerank_scores.get(doc, 0), reverse=True)

    return [list(doc) for doc in hashable_corpus[:int(len(corpus) * ratio)]]



def summarize(pdf1, ratio=0.1, word_count=None, split=False):

    output = ""

    text = get_text_pdf_full(pdf1)
    text = text.replace('\n','')
    ##txt_label = sent_pg_labeler(text)

    sentences = _clean_text_by_sentences(text)

    #input has to be at least MIN sentences

    corpus = _build_corpus(sentences)

    most_important_docs = summarize_corpus(corpus, ratio=ratio if word_count is None else 1)

    extracted_sentences = _extract_important_sentences(sentences, corpus, most_important_docs, word_count)

    extracted_sentences.sort(key=lambda s: s.index)

    str_temp = _format_results(extracted_sentences, split)

    temp_arr = str_temp.split('\n')
    for sent in temp_arr:
      pg = 1
      two_split_doc = text.split(sent)
      pg += two_split_doc[0].count("NEW_PAGE")
      output += (sent + ' (pg. ' + str(pg) + ')')
      output += '\n'

    output = output.replace('NEW_PAGE', '')
    lines = output.split('\n')
    regex = re.compile(r'.{1,80}(?:\s+|$)')
    output1 = '\n\n'.join(s.rstrip() for line in lines for s in regex.findall(line))
    text_file = open("pdf_summary.txt", "w+")
    n = text_file.write(output1)
    text_file.close()
type(summarize('samp_elec.pdf', 0.1))

NoneType

In [25]:
def doc_scorer(pdf1, score_words):
  score = 0
  doc = get_text_pdf_full(pdf1)
  doc = doc.replace('\n','')
  # score_words = ["electric vehicle", "EV", "electric vehicle supply equipment", "EVSE",
  #                "zero emission vehicle", "ZEV", "plug-in electric vehicle", "PEV", "own and operate",
  #                "cost recovery", "residential", "transit", "fleet", "school bus", "multi_family",
  #                "workplace", "public L2", "fast chargers", "DCFC", "charging stations", "EV chargers",
  #                "rebate", "third parties", "vendors", "EV software", "pilot program", "highway", "travel corridors"]
  for word in score_words:
    score += doc.count(word)
  std_score = score / len(score_words)
  return std_score

In [26]:
#TECO
doc_scorer('samp_elec.pdf')

TypeError: ignored

In [ ]:
#Duke Florida
doc_scorer('duke_florida.pdf')

In [ ]:
#Duke Carolinas
doc_scorer('duke_carolinas.pdf')

In [28]:
!pip install flask-ngrok

In [29]:
!export FLASK_ENV=development
from flask import Flask
from flask import *
from flask_ngrok import run_with_ngrok
from flask import render_template
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
# def upload():
#   return render_template("file_upload.html")
def home():
    return render_template("file_upload_final.html")

@app.route("/success_sum", methods=["POST", "GET"])
def suc_sum():
  global file
  f=request.files['file']
  suc_sum.file_name=f.filename
  f.save(suc_sum.file_name)
  suc_sum.file_out = summarize(suc_sum.file_name, 0.1)
  
  #name = suc_sum.file_out.split(".")[0] + "_summary.txt"
  return send_file("pdf_summary.txt", as_attachment = True)

@app.route("/success_score", methods=["POST", "GET"])
def suc_score():
  global file
  f=request.files['file']
  suc_score.file_name=f.filename
  f.save(suc_score.file_name)

  words = str(request.form["keywords"])
  words_list = words.split(",")
  suc_score.score = doc_scorer(suc_score.file_name, words_list)
  return render_template('file_upload_final.html', score = str(suc_score.score))
  #return send_file("pdf_summary.txt", as_attachment = True)

@app.route("/success_name", methods=["POST", "GET"])
def suc_name():
  global file
  f=request.files['file']
  suc_name.file_name=f.filename
  f.save(suc_name.file_name)

  suc_name.name = get_company_name(suc_name.file_name)
  return render_template('file_upload_final.html', name = str(suc_name.name))

@app.route("/success_date", methods=["POST", "GET"])
def suc_date():
  global file
  f=request.files['file']
  suc_date.file_name=f.filename
  f.save(suc_date.file_name)

  suc_date.date = get_date(suc_date.file_name)
  return render_template('file_upload_final.html', date = str(suc_date.date))
  #return send_file("pdf_summary.txt", as_attachment = True)

# def contact():
#     if request.method == 'POST':
#         if request.form['submit'] == 'Run Summary':
#             pass # do something
#         elif request.form['submit'] == 'Run Score':
#             pass # do something else
#         elif request.form['submit'] == 'Run Name':
#             pass # do something else
#         else:


if __name__ == "__main__":
  app.run()
    
#app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://efabc5ccf95a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Jul/2021 15:34:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2021 15:34:29] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# LABEL = "ANIMAL"
# TRAIN_DATA = [
#     (
#         "September 20, 2020", {"entities": [(0, 9, LABEL)]},
#     ),
#     (
#         "October 21, 2021", {"entities": [(0, 7, LABEL)]},
#     ),
#     (
#         "November 9, 2016 ", {"entities": [(0, 8, LABEL)]},
#     ),
#     (
#         "December 8, 2015", {"entities": [(0, 8, LABEL)]},
#     ),
#     (
#         "January 1, 2020", {"entities": [(0, 7, LABEL)]},
#     ),
#     (
#         "February 5, 2018", {"entities": [(0, 8, LABEL)]},
#     ),
#     (
#         "March 8, 2020", {"entities": [(0, 5, LABEL)]},
#     ),
#     (
#         "April 11, 2022", {"entities": [(0, 6, LABEL)]},
#     ),
#     (
#         "May 15, 2017", {"entities": [(0, 4, LABEL)]},
#     ),
#     (
#         "June 13, 2016", {"entities": [(0, 5, LABEL)]},
#     ),
#     (
#         "July 17, 2015", {"entities": [(0, 5, LABEL)]},
#     ),
#     (
#         "August 21, 2020", {"entities": [(0, 6, LABEL)]},
#     ),
#     (
#         "August 15, 2019", {"entities": [(0, 6, LABEL)]},
#     ),
#         (
#         "October 14, 2018", {"entities": [(0, 7, LABEL)]},
#     ),
#         (
#         "November 21, 2020", {"entities": [(0, 8, LABEL)]},
#     ),
#         (
#         "January 7, 2018", {"entities": [(0, 7, LABEL)]},
#     )
# ]


# nlp_d = spacy.blank("en")  # create blank Language class
# ner = nlp_d.create_pipe("ner")
# nlp_d.add_pipe(ner)

# ner.add_label(LABEL)  # add new entity label to entity recognizer

# optimizer = nlp_d.begin_training()

# move_names = list(ner.move_names)
# # get names of other pipes to disable them during training
# other_pipes = [pipe for pipe in nlp_d.pipe_names if pipe != "ner"]

# with nlp_d.disable_pipes(*other_pipes):  # only train NER
#     sizes = compounding(1.0, 4.0, 1.001)
#     # batch up the examples using spaCy's minibatch
#     for itn in range(n_iter):
#         random.shuffle(TRAIN_DATA)
#         batches = minibatch(TRAIN_DATA, size=sizes)
#         losses = {}
#         for batch in batches:
#             texts, annotations = zip(*batch)
#             nlp_d.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
#         print("Losses", losses)

In [ ]:
# !pip install streamlit
# #streamlit run app.py
# import streamlit as st
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# import maze

# st.title('Maze Solver')
# uploaded_file = st.file_uploader("Choose an image", ["jpg","jpeg","png"]) #image uploader
# st.write('Or')
# use_default_image = st.checkbox('Use default maze')